In [1]:
# Importamos las bibliotecas necesarias
import os
import pandas as pd
import boto3
from datetime import datetime
from botocore.exceptions import ClientError

In [2]:

# Definimos las constantes
BUCKET_NAME = "itam-analytics-ragp"
S3_RAW_DIRECTORY = "raw/"
PROFILE_NAME = "datascientist"
LOCAL_DATA_DIR = "data"

In [3]:
# Función para cargar los archivos CSV
def cargar_datos_locales():
    """
    Carga los archivos CSV con datos económicos desde el directorio local.
    """
    datos = {}
    
    # Buscamos los archivos CSV en el directorio local
    csv_files = {
        'tipo_cambio': 'tipo_cambio.csv',
        'inflacion': 'inflacion.csv',
        'tasas_interes': 'tasas_interes.csv'
    }
    
    for nombre, archivo in csv_files.items():
        ruta_completa = os.path.join(LOCAL_DATA_DIR, archivo)
        
        if os.path.exists(ruta_completa):
            try:
                datos[nombre] = pd.read_csv(ruta_completa)
                print(f"✅ Archivo {archivo} cargado: {len(datos[nombre])} registros")
            except Exception as e:
                print(f"❌ Error al cargar {archivo}: {str(e)}")
        else:
            print(f"❌ No se encontró el archivo {ruta_completa}")
    
    return datos


In [4]:
# Función para conectar a S3
def conectar_s3():
    """
    Establece conexión con AWS S3 usando el perfil especificado.
    """
    try:
        # Creamos una sesión usando el perfil
        session = boto3.Session(profile_name=PROFILE_NAME)
        # Creamos el cliente S3
        s3_client = session.client('s3')
        
        # Verificamos que el bucket existe
        response = s3_client.list_buckets()
        buckets = [bucket['Name'] for bucket in response['Buckets']]
        
        if BUCKET_NAME in buckets:
            print(f"✅ Conexión establecida con S3. Bucket '{BUCKET_NAME}' encontrado.")
            return s3_client
        else:
            print(f"⚠️ El bucket '{BUCKET_NAME}' no existe. Verifica el nombre.")
            return None
    
    except Exception as e:
        print(f"❌ Error al conectar con S3: {str(e)}")
        return None


In [5]:
# Función para verificar si un archivo existe en S3
def archivo_existe_en_s3(s3_client, s3_key):
    """
    Verifica si un archivo ya existe en S3.
    """
    try:
        s3_client.head_object(Bucket=BUCKET_NAME, Key=s3_key)
        return True
    except ClientError as e:
        # Si obtenemos un error 404, el archivo no existe
        if e.response['Error']['Code'] == '404':
            return False
        else:
            # Otro tipo de error
            print(f"❌ Error al verificar archivo en S3: {str(e)}")
            return False

In [6]:
# Función para subir los datos a S3
def subir_datos_a_s3(datos, s3_client):
    """
    Sube los DataFrames al bucket de S3 en el directorio raw,
    verificando primero si los archivos ya existen.
    """
    if not s3_client:
        print("❌ No se puede subir a S3 sin una conexión válida.")
        return False
    
    archivos_subidos = 0
    archivos_existentes = 0
    
    for nombre, df in datos.items():
        try:
            # Generamos la ruta en S3 (sin timestamp)
            s3_key = f"{S3_RAW_DIRECTORY}{nombre}/{nombre}.csv"
            
            # Verificamos si el archivo ya existe
            if archivo_existe_en_s3(s3_client, s3_key):
                print(f"⏩ Archivo ya existe en s3://{BUCKET_NAME}/{s3_key} - Omitiendo")
                archivos_existentes += 1
                continue
            
            # Si el archivo no existe, lo subimos
            csv_buffer = df.to_csv(index=False).encode()
            
            # Subimos el archivo
            s3_client.put_object(
                Bucket=BUCKET_NAME,
                Key=s3_key,
                Body=csv_buffer,
                ContentType='text/csv'
            )
            
            print(f"✅ Archivo subido a s3://{BUCKET_NAME}/{s3_key}")
            archivos_subidos += 1
            
        except Exception as e:
            print(f"❌ Error al subir {nombre}: {str(e)}")
    
    print(f"\n📤 Proceso completado:")
    print(f"   - {archivos_subidos} archivos nuevos subidos")
    print(f"   - {archivos_existentes} archivos ya existentes (omitidos)")
    print(f"   - {len(datos) - archivos_subidos - archivos_existentes} errores")
    
    return True


In [7]:
print(f"🔄 Iniciando proceso de carga de datos económicos a S3 - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
datos = cargar_datos_locales()
if not datos:
    print("❌ No se pudieron cargar datos. Abortando proceso ETL.")



🔄 Iniciando proceso de carga de datos económicos a S3 - 2025-03-14 16:47:51
✅ Archivo tipo_cambio.csv cargado: 240 registros
✅ Archivo inflacion.csv cargado: 241 registros
✅ Archivo tasas_interes.csv cargado: 240 registros


In [8]:

print("\n📤 FASE DE CARGA")
print("---------------")
s3_client = conectar_s3()
exito = subir_datos_a_s3(datos, s3_client)

# Resumen final
if exito:
    print("\n✅ PROCESO ETL COMPLETADO EXITOSAMENTE")
else:
    print("\n⚠️ PROCESO ETL COMPLETADO CON ERRORES")


📤 FASE DE CARGA
---------------
✅ Conexión establecida con S3. Bucket 'itam-analytics-ragp' encontrado.
✅ Archivo subido a s3://itam-analytics-ragp/raw/tipo_cambio/tipo_cambio.csv
✅ Archivo subido a s3://itam-analytics-ragp/raw/inflacion/inflacion.csv
✅ Archivo subido a s3://itam-analytics-ragp/raw/tasas_interes/tasas_interes.csv

📤 Proceso completado:
   - 3 archivos nuevos subidos
   - 0 archivos ya existentes (omitidos)
   - 0 errores

✅ PROCESO ETL COMPLETADO EXITOSAMENTE
